In [1]:
#第七章 数据清洗和准备
#在数据分析和建模的过程中，相当多的时间要用在数据准备上：加载、清理、转换以及重塑
#本章中，会讨论处理确实数据、重复数据、字符串操作和其他分析数据转换的工具

In [2]:
#7.1处理缺失数据
#在许多数据分析工作中，缺失数据是经常发生的。pandas的目标之一就是尽量轻松地处理缺失数据
#例如，pandas对象的所有描述性统计默认都不包括缺失数据
#对于数值数据，pandas使用浮点值NaN（not a number）表示缺失数据
import pandas as pd
import numpy as np
string_data=pd.Series(['aardvark','artichoke',np.nan,'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
#在pandas中，采用R语言的惯用法，即将缺失值表示为NA
#当进行数据清洗以进行分析时，最好直接对缺失数据进行分析，以判断数据采集的问题或缺失数据可能导致的偏差
#python内置的None值在对象数组中也可以作为NA
string_data[0]=None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

In [5]:
#滤除缺失数据
from numpy import nan as NA
data=pd.Series([1,NA,3.5,NA,7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [6]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [7]:
#这等价于
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [8]:
data.notnull()

0     True
1    False
2     True
3    False
4     True
dtype: bool

In [9]:
#而对于DataFrame对象，事情就有点复杂了。可能希望全丢弃NA或含有NA的行或列
#dropna丢弃任何含有缺失值的行
data=pd.DataFrame([[1.,6.5,3.],[1.,NA,NA],[NA,NA,NA],[NA,6.5,3.]])
cleaned=data.dropna()

In [10]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [12]:
#传入how=‘all’将只丢弃全为NA的那些行：
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [13]:
data[4]=NA
data[5]=NA
data

,0,1,2,4,5
0,1.0,6.5,3.0,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN,NaN


In [14]:
#用这种方式丢弃列，只需传入axis=1即可
data.dropna(axis=1,how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [15]:
#另一个滤除DataFrame行的问题涉及时间序列数据，假设只想留下一部分观测数据，可以用thresh参数实现此目的
df=pd.DataFrame(np.random.randn(7,3))
df.iloc[:4,1]=NA
df

,0,1,2
0,0.334732,NaN,1.188958
1,0.102922,NaN,-0.448902
2,-0.965319,NaN,-0.104842
3,0.519785,NaN,-1.779687
4,0.064440,-0.255824,1.255877
5,1.114041,-1.282383,-1.147024
6,0.021914,0.865379,-0.306580


In [16]:
df.iloc[:2,2]=NA
df

,0,1,2
0,0.334732,NaN,NaN
1,0.102922,NaN,NaN
2,-0.965319,NaN,-0.104842
3,0.519785,NaN,-1.779687
4,0.064440,-0.255824,1.255877
5,1.114041,-1.282383,-1.147024
6,0.021914,0.865379,-0.306580


In [17]:
df.dropna()

,0,1,2
4,0.064440,-0.255824,1.255877
5,1.114041,-1.282383,-1.147024
6,0.021914,0.865379,-0.306580


In [18]:
df.dropna(thresh=1)

,0,1,2
0,0.334732,NaN,NaN
1,0.102922,NaN,NaN
2,-0.965319,NaN,-0.104842
3,0.519785,NaN,-1.779687
4,0.064440,-0.255824,1.255877
5,1.114041,-1.282383,-1.147024
6,0.021914,0.865379,-0.306580


In [19]:
#填充缺失数据
#可能不想滤除缺失数据,而希望通过其他方式填补那些‘空洞’。
#对于大多数情况下，fillna方法是最主要的函数。通过一个常数调用fillna就会将缺失值替换为那个常数值
df.fillna(0)

,0,1,2
0,0.334732,0.000000,0.000000
1,0.102922,0.000000,0.000000
2,-0.965319,0.000000,-0.104842
3,0.519785,0.000000,-1.779687
4,0.064440,-0.255824,1.255877
5,1.114041,-1.282383,-1.147024
6,0.021914,0.865379,-0.306580


In [20]:
#若是通过一个字典调用fillna，就可以实现对不同的列填充不同的值：
df.fillna({2:0.5,1:0})

,0,1,2
0,0.334732,0.000000,0.500000
1,0.102922,0.000000,0.500000
2,-0.965319,0.000000,-0.104842
3,0.519785,0.000000,-1.779687
4,0.064440,-0.255824,1.255877
5,1.114041,-1.282383,-1.147024
6,0.021914,0.865379,-0.306580


In [21]:
#fillna默认会返回新对象，但也可以对现有对象进行就地修改：
_=df.fillna(0,inplace=True)
df

,0,1,2
0,0.334732,0.000000,0.000000
1,0.102922,0.000000,0.000000
2,-0.965319,0.000000,-0.104842
3,0.519785,0.000000,-1.779687
4,0.064440,-0.255824,1.255877
5,1.114041,-1.282383,-1.147024
6,0.021914,0.865379,-0.306580


In [22]:
#对reindexing有效的那些插值方法也可用于fillna
df=pd.DataFrame(np.random.randn(6,3))
df.iloc[2:,1]=NA
df

,0,1,2
0,1.818121,0.653041,0.740271
1,0.904017,1.916545,0.118105
2,-2.132453,NaN,0.522613
3,0.041958,NaN,0.004528
4,-0.891477,NaN,0.300086
5,0.709114,NaN,1.045184


In [23]:
df.iloc[4:,2]=NA
df

,0,1,2
0,1.818121,0.653041,0.740271
1,0.904017,1.916545,0.118105
2,-2.132453,NaN,0.522613
3,0.041958,NaN,0.004528
4,-0.891477,NaN,NaN
5,0.709114,NaN,NaN


In [24]:
df.fillna(method='ffill')

,0,1,2
0,1.818121,0.653041,0.740271
1,0.904017,1.916545,0.118105
2,-2.132453,1.916545,0.522613
3,0.041958,1.916545,0.004528
4,-0.891477,1.916545,0.004528
5,0.709114,1.916545,0.004528


In [25]:
df.fillna(method='ffill',limit=2)

,0,1,2
0,1.818121,0.653041,0.740271
1,0.904017,1.916545,0.118105
2,-2.132453,1.916545,0.522613
3,0.041958,1.916545,0.004528
4,-0.891477,NaN,0.004528
5,0.709114,NaN,0.004528


In [26]:
#传入Series的平均值或中位数
data=pd.Series([1.,NA,3.5,NA,7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

In [27]:
#7.2数据转换
#数据的重排，另一类重要操作则时过滤、清洗以及其他的转换工作
#移除重复数据
data=pd.DataFrame({'k1':['one','two']*3+['two'],'k2':[1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [28]:
#dataFrame的duplicated方法返回一个布尔型Series，表示各行是否是重复行
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [29]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [30]:
#这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。
data['v1']=range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [31]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [32]:
#duplicated和drop_duplicates默认保留的是第一个出现的值的组合
#传入keep=‘last’则保留最后一个
data.drop_duplicates(['k1','k2'],keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


In [33]:
#利用函数或映射进行数据转换
#对于许多数据集，可能希望根据数组、Series或DataFrame列中的值来实现转换工作
data=pd.DataFrame({'food':['bacon','pulled pork','bacon','Pastrami','corned beef','Bacon','pastrami','honey ham','novalox'],
                  'ounces':[4,3,12,6,7.5,8,3,5,6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,novalox,6.0


In [34]:
meat_to_animal={'bacon':'pig','pulled pork':'pig','pastrami':'cow','corned beef':'cow','honey ham':'pig','novalox':'salmon'}
meat_to_animal

{'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'novalox': 'salmon'}

In [35]:
lowercased=data['food'].str.lower()

In [36]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8        novalox
Name: food, dtype: object

In [37]:
data['animal']=lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,novalox,6.0,salmon


In [38]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [39]:
#替换值
#利用fillna方法填充缺失数据可以看作值替换的一种特殊情况
#map可修改对象的数据子集，而replace则提供了一种实现该功能的更简单、更灵活方式
data=pd.Series([1.,-999.,2.,-999.,-1000.,3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [40]:
#-999这个值可能是一个表示缺失数据的标记值。
#要将其替换为pandas能够理解的NA值。可以利用replace来产生一个新的Series（除非传入inplace=True）
data.replace(-999,np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [41]:
#要让每个值有不同的替换值，可以传递一个替换列表
data.replace([-999,-1000],[np.nan,0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [42]:
#传入的参数也可以是字典
data.replace({-999:np.nan,-1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [43]:
#重命名轴索引
#跟Series中的值一样，轴标签也可以通过函数或映射进行转换。从而得到一个新的不同标签的对象
data=pd.DataFrame(np.arange(12).reshape((3,4)),index=['Ohio','Colorado','New York'],columns=['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [44]:
#跟Series一样，轴索引也有一个map方法
transform=lambda x:x[:4].upper()

In [45]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [46]:
#你可以将其赋值给index，这样就可以对DataFrame进行就地修改
data.index=data.index.map(transform)

In [47]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [48]:
#如果想要创建数据集的转换版，比较实用的方法是rename
data.rename(index=str.title,columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [49]:
#rename可以结合字典型对象实现对部分轴标签的更新：
data.rename(index={'OHIO':'INDIANA'},columns={'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [50]:
#如果希望就地修改某个数据集，传入inplace=True即可
data.rename(index={'OHIO':'INDIANA'},inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [51]:
#离散化和面元划分
#为了便于分析，连续数据常常被离散化或拆分为‘面元’。假设有一组人员数据，而你希望将它们划分为不同的年龄组：
ages=[20,22,25,27,21,23,37,31,61,45,41,32]
#接下来，将这些数据划分为‘18到25’、‘26到35’、'35到60'以及’60以上‘几个面元。要实现该功能，需要使用pandas的cut函数
bins=[18,25,35,60,100]
cats=pd.cut(ages,bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [52]:
#pandas返回的是一个特殊的categorical对象。结果展示了pandas.cut划分的面元。
#你可以将其看作一组表示面元名称的字符串。它的底层含有一个表示不同分类名称的类型数组，以及一个codes属性中年龄数据的标签
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [53]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [54]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [55]:
#跟‘区间’的数学符号一样，圆括号表示开端，放括号表示闭端，哪边是闭端可以通过right=False进行修改
pd.cut(ages,[18,26,36,61,100],right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [56]:
#可以通过传递一个列表或数组到lables，设置自己的面元名称：
group_names=['Youth','YoungAdult','MiddleAged','Senior']
pd.cut(ages,bins,labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [57]:
#如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算等长面元
data=np.random.randn(20)
pd.cut(data,4,precision=2)

[(-1.68, -0.83], (0.85, 1.69], (-1.68, -0.83], (0.85, 1.69], (-1.68, -0.83], ..., (0.0099, 0.85], (0.85, 1.69], (-1.68, -0.83], (-0.83, 0.0099], (0.0099, 0.85]]
Length: 20
Categories (4, interval[float64]): [(-1.68, -0.83] < (-0.83, 0.0099] < (0.0099, 0.85] < (0.85, 1.69]]

In [58]:
#qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。根据数据的分布情况，cut可能无法使各个面元中
#含有相同数量的数据点。而qcut由于使用的是样本分位数，因此可以得到大小基本相同的面元
data=np.random.randn(1000)
cats=pd.qcut(data,4)
cats

[(-2.983, -0.614], (0.0798, 0.779], (0.779, 3.118], (-2.983, -0.614], (0.0798, 0.779], ..., (-2.983, -0.614], (-2.983, -0.614], (-2.983, -0.614], (0.779, 3.118], (-0.614, 0.0798]]
Length: 1000
Categories (4, interval[float64]): [(-2.983, -0.614] < (-0.614, 0.0798] < (0.0798, 0.779] < (0.779, 3.118]]

In [59]:
pd.value_counts(cats)

(0.779, 3.118]      250
(0.0798, 0.779]     250
(-0.614, 0.0798]    250
(-2.983, -0.614]    250
dtype: int64

In [60]:
#与cut类似，也可以传递自定义的分位数（0到1之间的数值，包含端点）
pd.qcut(data,[0,0.1,0.5,0.9,1.])

[(-2.983, -1.256], (0.0798, 1.344], (1.344, 3.118], (-1.256, 0.0798], (0.0798, 1.344], ..., (-1.256, 0.0798], (-1.256, 0.0798], (-1.256, 0.0798], (0.0798, 1.344], (-1.256, 0.0798]]
Length: 1000
Categories (4, interval[float64]): [(-2.983, -1.256] < (-1.256, 0.0798] < (0.0798, 1.344] < (1.344, 3.118]]

In [61]:
#cut和qcut这两个离散化函数对分位和分组分析非常重要

In [62]:
#检测和过滤异常值
#过滤或变换异常值在很大程度上就是运用数组运算
data=pd.DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.040728,-0.027920,-0.011942,-0.001167
std,0.970066,0.989162,0.988307,1.001165
min,-2.924160,-2.913050,-3.721728,-3.147192
25%,-0.620526,-0.736818,-0.665979,-0.668667
50%,0.057037,-0.006261,-0.064262,-0.010641
75%,0.709118,0.668988,0.661130,0.697102
max,2.668688,3.307794,2.898275,3.206161


In [63]:
data

,0,1,2,3
0,1.177565,-1.085254,1.289457,-0.340620
1,2.604869,-0.382229,0.916773,-1.097028
2,-0.277088,-1.522773,-0.961367,0.400828
3,0.562749,-0.385523,0.863864,0.696153
4,0.233288,-0.554724,1.373180,0.627405
5,-0.879038,0.234322,-0.913112,-1.065212
6,-1.302756,-0.418015,0.981362,0.928138
7,0.277474,0.078841,0.495329,-0.166932
8,1.812892,1.626877,0.782685,-0.271012
9,1.306676,-0.720957,0.271557,-0.496617


In [64]:
#假设你想要找出某列中绝对值大小超过3的值
col=data[2]
col[np.abs(col)>3]

632   -3.070184
759   -3.721728
Name: 2, dtype: float64

In [65]:
#要选择出全部含有‘超过3或-3的值’的行，你可以在布尔型DataFrame中使用any方法
data[(np.abs(data)>3).any(1)]

,0,1,2,3
61,-1.319928,-1.078027,-0.129595,-3.077896
122,0.667538,0.444457,0.937572,-3.147192
165,0.795698,0.400320,-0.057173,3.011671
261,-1.964653,-0.732047,-1.396598,-3.003123
482,-0.715370,0.345505,1.277369,3.206161
498,-0.037393,3.307794,0.225812,-0.368010
632,1.471083,-0.339915,-3.070184,1.125338
759,0.244322,0.928663,-3.721728,1.005201


In [66]:
data[np.abs(data)>3]= np.sign(data)*3

In [67]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,-1.0
1,1.0,-1.0,1.0,-1.0
2,-1.0,-1.0,-1.0,1.0
3,1.0,-1.0,1.0,1.0
4,1.0,-1.0,1.0,1.0


In [68]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.040728,-0.028228,-0.011150,-0.001157
std,0.970066,0.988171,0.985640,0.999791
min,-2.924160,-2.913050,-3.000000,-3.000000
25%,-0.620526,-0.736818,-0.665979,-0.668667
50%,0.057037,-0.006261,-0.064262,-0.010641
75%,0.709118,0.668988,0.661130,0.697102
max,2.668688,3.000000,2.898275,3.000000


In [69]:
#根据数据的值是正还是负，np.sign(data)可以生成1和-1
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,-1.0
1,1.0,-1.0,1.0,-1.0
2,-1.0,-1.0,-1.0,1.0
3,1.0,-1.0,1.0,1.0
4,1.0,-1.0,1.0,1.0


In [70]:
#排序和随机采样
#利用numpy.random.permutation函数可以轻松实现对Series或DataFrame的列的排列工作
#通过需要排列的轴的长度调用permutation,可产生一个表示新顺序的整数数组
df=pd.DataFrame(np.arange(5*4).reshape((5,4)))
sampler=np.random.permutation(5)
sampler

array([2, 4, 0, 3, 1])

In [71]:
#基于iloc的索引操作或take函数中使用该数组了
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [72]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15
1,4,5,6,7


In [73]:
#如果不想用替换的方式选取随机子集，可以在Series和DataFrame上使用sample方法
df.sample(n=3)

,0,1,2,3
3,12,13,14,15
0,0,1,2,3
4,16,17,18,19


In [74]:
#要通过替换的方式产生样本（允许重复选择），可以传递replace=True到sample
choices=pd.Series([5,7,-1,6,4])
draws=choices.sample(n=10,replace=True)
draws

0    5
3    6
3    6
3    6
4    4
1    7
2   -1
4    4
0    5
4    4
dtype: int64

In [75]:
#计算指标/哑变量
#另一种常用于统计建模或机器学习的转换方式是：将分类变量转换为‘哑变量’或‘指标矩阵’

In [76]:
#如果DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame。pandas有一个get_dummies函数可以实现该功能

In [77]:
df=pd.DataFrame({'key':['b','b','a','c','a','b'],'data1':range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [78]:
#有时候，可能想给指标DataFrame的列加上一个前缀，以便和其他数据进行合并。get_dummies的prefix参数可以实现该功能
dummies=pd.get_dummies(df['key'],prefix='key')
df_with_dummy=df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [79]:
mnames=['movie_id','title','genres']

In [80]:
movies=pd.read_table(r'C:\Users\86139\Desktop\examples_data_for_data_analysis\movies.dat',sep='::',header=None,names=mnames)

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [81]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [82]:
all_genres=[]
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres=pd.unique(all_genres)

In [83]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [84]:
#构建指标DataFrame的方法之一是从一个全零DataFrame开始
zero_matrix=np.zeros((len(movies),len(genres)))
dummies=pd.DataFrame(zero_matrix,columns=genres)
gen=movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [85]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [86]:
#根据索引，使用.loc设定值
for i,gen in enumerate (movies.genres):
    indices=dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i,indices]=1

In [87]:
movies_windic=movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [88]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [89]:
#对于很大的数据，用这种方式构建多成员指标变量就会变得非常慢。最好使用更低级的函数，将其写入Numpy数组，然后结果包装在
#Dataframe中
#一个对统计应用有用的秘诀是：结合get_dummies和诸如cut之类的离散化函数
np.random.seed(123456)
values=np.random.rand(10)

In [90]:
values

array([0.12696983, 0.96671784, 0.26047601, 0.89723652, 0.37674972,
       0.33622174, 0.45137647, 0.84025508, 0.12310214, 0.5430262 ])

In [91]:
bins=[0,0.2,0.4,0.6,0.8,1]

In [92]:
pd.get_dummies(pd.cut(values,bins))

   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           1           0           0           0           0
1           0           0           0           0           1
2           0           1           0           0           0
3           0           0           0           0           1
4           0           1           0           0           0
5           0           1           0           0           0
6           0           0           1           0           0
7           0           0           0           0           1
8           1           0           0           0           0
9           0           0           1           0           0

In [93]:
#7.3字符串操作
#字符串对象方法
val='a,b,guido'
val.split(',')

['a', 'b', 'guido']

In [94]:
#split常常与strip一起使用，以去除空白符
pieces=[x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [95]:
#利用加法，可以将这些子字符串以双冒号分隔符的形式连接起来：
first,second,third=pieces

In [96]:
first+'::'+second+'::'+third

'a::b::guido'

In [97]:
'::'.join(pieces)

'a::b::guido'

In [98]:
'guido' in val

True

In [99]:
val.index(',')

1

In [100]:
val.find(':')

-1

In [101]:
val.count(',')

2

In [102]:
val.replace(',','::')

'a::b::guido'

In [103]:
val.replace(',','')

'abguido'

In [104]:
#正则表达式
#re模块函数可以分为三个大类：模式匹配、替换以及拆分。
#一个regex描述了需要在文本中定位的一个模式。
import re

In [105]:
text='foo   bar\t baz  \tqux'

In [106]:
re.split('\s+',text)

['foo', 'bar', 'baz', 'qux']

In [107]:
#调用re.split('\s+,text')时，正则表达式回先被编译，然后再再text上调用其split方法
#用re.compile自己编译regex以得到一个可重用的regex对象
regex=re.compile('\s+')

In [108]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [109]:
#如果只希望得到匹配regex的所有模式，则可以使用findall方法：
regex.findall(text)

['   ', '\t ', '  \t']

In [110]:
#识别大部分的电子邮件地址的正则表达式
text="""Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern=r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [111]:
regex=re.compile(pattern,flags=re.IGNORECASE)

In [112]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [113]:
#search返回的是文本中第一个电子邮箱地址（以特殊的匹配项对象形式返回）
m=regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [114]:
text[m.start():m.end()]

'dave@google.com'

In [115]:
print(regex.match(text))

None


In [116]:
print(regex.sub('REDACTED',text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [117]:
#假设不仅想要找出电子邮件地址，还想将各个地址分成3个部分:用户名、域名以及域后缀。
#要实现此功能，只需将待分段的模式的各部分用圆括号包起来即可
pattern=r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [118]:
regex=re.compile(pattern,flags=re.IGNORECASE)

In [119]:
m=regex.match('wesm@bright.net')

In [120]:
m.groups()

('wesm', 'bright', 'net')

In [121]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [122]:
print(regex.sub(r'Username:\1,Domain:\2,Suffix:\3',text))

Dave Username:dave,Domain:google,Suffix:com
Steve Username:steve,Domain:gmail,Suffix:com
Rob Username:rob,Domain:gmail,Suffix:com
Ryan Username:ryan,Domain:yahoo,Suffix:com



In [123]:
#pandas的矢量化字符串函数
#清理待分析的散乱数据时，常常需要做一些字符串规整化工作
data={'Dave':'dave@google.com','Steve':'steve@gmail.com','Rob':'rob@gmail.com','Wes':np.nan}


In [124]:
data=pd.Series(data)

In [125]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [126]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [127]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [128]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [129]:
data.str.findall(pattern,flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [130]:
#有两个办法可以实现矢量化的元素获取操作：要么使用str.get，要么在str属性上使用索引：
matches=data.str.match(pattern,flags=re.IGNORECASE)

In [131]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [132]:
#pandas的矢量化字符串函数
#清理待分析的散乱数据时，常常需要做一些字符串规整化工作。更为复杂的情况时，含有字符串的列有时还含有缺失数据
data={'Dave':'dave@google.com','Steve':'steve@gmail.com','Rob':'rob@gmail.com','Wes':np.nan}
data=pd.Series(data)

In [133]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [134]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [135]:
#通过data.map,所有字符串和正则表达式方法都能被应用于（传入lambda表达式或其他函数）各个值，但是如果存在NA（null）就会报错。
#为了解决这个问题，Series有一些能够跳过NA值得面向数组方法，进行字符串操作。
#通过Series得str属性即可访问这些方法
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [136]:
#也可以使用正则表达式，还可以加上任意re选项（如IGNORECASE）
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [137]:
data.str.findall(pattern,flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [139]:
#有两个办法可以实现矢量化得元素获取操作:要么使用str.get,要么在str属性上使用索引:
matches=data.str.match(pattern,flags=re.IGNORECASE)

In [140]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [141]:
#要访问嵌入列表中的元素，我们可以传递索引到这两个函数中：
matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [142]:
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [143]:
#对字符串进行截取
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [ ]:
#7.4总结
#下一章，我们会学习pandas的聚合与分组